In [ ]:
# Import needed stuff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from config import census_api_key
from census import Census

In [ ]:
# Get the data from the MDE data available on MDE's website
MDE_data = pd.read_csv("MDE_Data.csv")
MDE_data

In [ ]:
census_response = []

# Get the census population and poverty data for school districts in minnesota
url = f"https://api.census.gov/data/2018/acs/acs5?get=NAME,B01003_001E,B17001_002E&for=school%20district%20(unified):*&in=state:27&key={census_api_key}"
try:
    census_response = requests.get(url).json()
except:
    pass

# Put results in a data from and drop the first row with headers
census_response_df = pd.DataFrame(census_response)
census_response_df = census_response_df.drop([0])

# Rename the columns
census_response_df = census_response_df.rename(columns = {0:"School District", 1:"Population", 2:"Poverty", 3: "State Number", 4:"Census School Number"})

# Strip off Minnesota from school district names so we can merge with MDE data
census_response_df["School District"] = census_response_df["School District"].str.replace(', Minnesota', '')

# Just pull the data we are interested in
clean_census_data = census_response_df.loc[:,["School District", "Population", "Poverty", "Census School Number"]]
clean_census_data.to_csv("Census_data.csv", index=False)
clean_census_data

In [ ]:
# Merge the two data frames on the school district name.
merged_census_MDE_df = pd.merge(MDE_data, clean_census_data, right_on = 'School District', left_on = 'District Name')
merged_census_MDE_df.to_csv("Merged_MDE_Census_data.csv", index=False)
merged_census_MDE_df

In [ ]:
# Part II Start

In [ ]:
# Part II End

In [ ]:
# Part III Start

In [ ]:
# Part III End

In [ ]:
# Part IV Start

In [ ]:
# Part IV End